In [367]:
import iris

import numpy as np
import scipy.stats

In [368]:
%load_ext autoreload
%autoreload 2
import math_helpers

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [369]:
era5 = iris.load("testing_data/ERA5.nc")
obs = era5[1]

cm_hist = iris.load_cube(
    "testing_data/CMIP6/medium_area/historical_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_historical_r1i1p1f1_gr_19790116-20141216_v20200729.nc"
)
cm_future = iris.load_cube(
    "testing_data/CMIP6/medium_area/SSP3-7_monthly_surface_temp_single_levels_IPSL-CM5A2-INCA/tas_Amon_IPSL-CM5A2-INCA_ssp370_r1i1p1f1_gr_20150116-20501216_v20201218.nc"
)

/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(
/home/jakobwes/anaconda3/lib/python3.8/site-packages/iris/fileformats/cf.py:859: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tas'
  warnings.warn(


In [370]:
# Regrid
cm_hist = cm_hist.regrid(obs, iris.analysis.Linear())
cm_future = cm_future.regrid(obs, iris.analysis.Linear())

In [371]:
# Get data
obs = np.array(obs.data[:, 0, :, :])
cm_hist = cm_hist.data
cm_future = cm_future.data

Test ISIMIP:

In [372]:
obs = obs[:, 1, 1]
cm_hist = cm_hist[:, 1, 1]
cm_future = cm_future[:, 1, 1]

In [386]:
from isimip import ISIMIP

temp_debiaser = ISIMIP(variable="tas")
temp_debiaser.variable = "prsnratio"
"""temp_debiaser.trend_preservation = "mixed"
temp_debiaser.lower_bound = 0
temp_debiaser.upper_bound = 300
temp_debiaser.lower_threshold = 50
temp_debiaser.upper_treshold = 290
temp_debiaser.powerlaw_exponent_step4 = 2"""

'temp_debiaser.trend_preservation = "mixed"\ntemp_debiaser.lower_bound = 0\ntemp_debiaser.upper_bound = 300\ntemp_debiaser.lower_threshold = 50\ntemp_debiaser.upper_treshold = 290\ntemp_debiaser.powerlaw_exponent_step4 = 2'

In [387]:
obs = obs[obs > 0]  # Two obs-outliers. Need to check for that.

In [388]:
obs, cm_hist, cm_future = temp_debiaser.step2(obs, cm_hist, cm_future)
obs, cm_hist, cm_future = temp_debiaser.step4(obs, cm_hist, cm_future)
obs_future = temp_debiaser.step5(obs, cm_hist, cm_future)
test = temp_debiaser.step6(obs, obs_future, cm_hist, cm_future)

In [389]:
x = temp_debiaser.apply_location(obs, cm_hist, cm_future)

In [390]:
test

array([278.08251007, 276.54369091, 280.84551741, 280.95059937,
       287.33865147, 291.42818074, 292.12039497, 292.85133194,
       289.35639597, 285.7120347 , 283.15117404, 278.08022658,
       277.64277361, 277.98877754, 280.22402958, 282.72725741,
       285.44909488, 292.27991226, 293.66002576, 293.12135856,
       290.84189422, 288.00544842, 281.55980711, 280.14262354,
       271.26760629, 279.4966137 , 282.7343968 , 281.34495123,
       284.56846286, 289.78292192, 293.91177287, 294.64476497,
       289.25857037, 284.69406321, 281.35874737, 278.79741402,
       278.11513672, 279.26613774, 282.69694454, 280.42316015,
       286.53672482, 291.35072885, 290.20047761, 293.00295585,
       290.47291226, 283.74187791, 282.15594509, 280.14107434,
       279.88872803, 278.46700366, 281.83505682, 283.13324505,
       288.05257127, 293.21647153, 292.91303391, 291.61625684,
       291.94938292, 286.27375551, 282.82373855, 279.94094985,
       277.83630526, 280.72409114, 282.13944641, 286.19

Missing from ISIMIP: 

- Steps 1 and 8 (scaling for rsds)
- Steps 2 and 7 (trend removal)
- Code-consistency, user-options, non-parametric, interface, etc.